# Q1 - Implement the mathematics of complex numbers in Python (40 points)

---

Recall that a complex number has real and imaginary parts, x + iy; where x is the real part and y is the imaginary part. Define a "Complex" class in Python that can be used as follows

c1 = Complex(1,2)

c2 = Complex(2,3)

print(c1 + c2) # prints "3 + 5i"

print(c1 - c2) # prints "-1 - i"

print(c1 * c2) # prints "-4 + 7i"

print(c1 / c2) # prints "0.61538461538 + 0.07692307692i" (optional)

print(c1.abs()) # prints "2.23606798"


Look up definitions of division/absolute value if you need to

# Q2 - Implement a polynomial computer in Python (60 points)

---

A polynomial in Python can be represented by a list. For example, 


fx = [-2, 0, 1]


represents the polynomial x^2 - 2 = 0. i.e. The elements of the list correspond to the coefficients of the polynomial. Implement a "Polynomial" class in Python that can be used as follows


fx = Polynomial([-2, 0, 1])

print("Solving f(x) =", fx)

print('Answer \t =', fx.solve(1)) # should print 1.414...

That is, implement a "solve" method, that takes as input an initial value for x, and returns the solution of that polynomial using Newton's Method.


Note - For both these questions, start defining the class and __init__, and __str__ methods. You may define any helper methods that you need.


---

Change the comments in the main() to allow user input.

In [73]:
import re
import math


class Polynomial:
    """Defines a polynomial function and the opperations that can be performed on and with it.\n\n
    
    polynomial( *args , name = function , dependent_var = y , independent_var = x).\n\n
    
    \t*args - ( list of ints) - The coefficients of the polynomial. ie: f(x) = 2x^2+5x-12 -> fu = polynomial([ 2 , 5 , -12 ]).\n
    \tname - (str) - Ex: g(s) The name of the function that will be displayed when called by print(). For primes to format correctly, add brackets around independent variable. Defaults to f(x) \n
    \tdependent_var - (str) - The designator for the dependent variable. Defaults to y.\n
    \tindependent_var - (str) - The designator for the independent variable. Defaults to x.\n\n
    
    Methods:\n
    \tpolynomial.compute( computer = 'n' , precision = 4 , variable = 'i').\n

    \tpolynomial.derive()
    """
    
    # constructor / initlization method
    def __init__(self, coefficients = 0, name = 'f(x)' , dependent_var = 'y' , independent_var = 'x'):
        """Defines the original polynomial and the derrivitive therof"""

        self.terms = coefficients[::-1] # This list is reversed in order to make the nth index = the power that the coefficient is being raised to. I decided to do it this way to make the user input more natural.
        self.name = name
        self.dep_var = dependent_var
        self.ind_var = independent_var

    #str definition method.
    def __str__(self):
        """Defines now the class is to be printed or used as a string"""

        # Add function name header.
        formatted_string = f'{self.name} = '
        
        #We will need to reverse the list of terms to get it into standard form, this empty list will collect the formatted terms.
        formatted_terms = []

        #We need a separate formatting for positive and negative terms.
        for power , coefficient in enumerate(self.terms):
            
            # Case for first term, to not add an unnecesarry + to output.
            if power == len(self.terms) - 1 and len(self.terms) > 1:
                formatted_terms.append(
                    join_string([' ', re.sub(f'[-]' , ' - ' , str(coefficient)), self.ind_var , '^', str(power)]))

            # Case for last term, does not need ^0 or operator if only one term long   
            elif power == 0:
                if coefficient > -1 and len(self.terms) > 1:
                    formatted_terms.append(
                        join_string([' + ', str(coefficient)]))
                else:
                    formatted_terms.append(
                        join_string([' - ' , re.sub(f'[-]' , '' , str(coefficient))]))               
            
            #Default case, needs + if positive.
            else:
                if coefficient > -1:
                    formatted_terms.append(
                        join_string([' + ', str(coefficient), self.ind_var , '^', str(power)]))
                else:
                    formatted_terms.append(
                        join_string([' - ' , re.sub(f'[-]' , '' , str(coefficient)) , self.ind_var , '^' , str(power)]))
        
        return formatted_string + join_string(formatted_terms[::-1])

    #Uses the defined function to compute for y with a given x.
    def compute( self , independent_var ):
        computed_terms = 0
        for power , coefficient in enumerate(self.terms):
            computed_terms += coefficient * pow( independent_var , power )

        new_expression = re.sub(str(self.ind_var) , str(independent_var), self.name)

        return [f'{new_expression} = {computed_terms}' , computed_terms]

    # Polynomial solver method.
    def solve(self, starting_guess = 1 , precision = 4 ):
        """This function uses Newton's method to aproximate the x-intercept of a function.\n
        \tinitial_guess - ( int , float ) - The starting position of the"""

        #This is x(n)
        x_n = starting_guess

        # Create a new polynomial that is f'. We will ise this repeatedly later *spoilers*.
        f_prime = self.derive()

        while True:
            
            if f_prime.compute(x_n)[1] == 0:
                return f'Division by 0 error > solve failed'

            # Find next approximation.
            x_n_plus_1 = x_n - (self.compute(x_n)[1] / f_prime.compute(x_n)[1])

            # Check if the two consecutive approximations are close enough.
            if math.isclose(x_n_plus_1, x_n, abs_tol = pow( 10 , -precision )):
                break

            # Update the guess for the next iteration.
            x_n = x_n_plus_1

        return [ f'The root found of {self.name} is : {round(x_n_plus_1 , precision )}' , x_n_plus_1]



    # Polynomial derive method ouputs a new polynomial.
    def derive(self):
        """This method derives the polynomial and returns a new polynomial"""
        if len(self.terms) > 1:
            # The reason for the reversal of order in ennumerate is because of the descision I made to have the polynomial class take the coefficients in left to right order on init.
            return Polynomial(
                [ coefficient * (power + 1) for power , coefficient in enumerate(self.terms[1::])][::-1] ,
                  re.sub(f'[(]' , "'(" , self.name ) , self.dep_var , self.ind_var ) # The sub will turn f'(x) into f''(x) etc... 
        else:
            return Polynomial( [0] , re.sub(f'[(]' , "'(" , self.name ) , self.dep_var , self.ind_var)



def join_string(string_list):
    """Takes a list as an argument and returns a single joined string"""

    return ''.join(string for string in string_list)


def usr_input():
    """Collects user inputs and validates input them so they can be used by the polynomial class. 
    
    Returns a list of integer coefficients"""

    # Empty list that will store the coefficients.
    all_coefficients = [] 

    while True:
        usr_entry = input('Enter a coefficient (or "x" to finish) >>> ').strip()

        # If the cleaned input is 'x' or 'X', exit the loop.
        if usr_entry.lower() == 'x':
            break

        # If the input matched the form "optional -" followed by integers, add it to the list.
        if re.fullmatch(r'-?\d+', usr_entry):
            #convert last input to an integer and add it to the list.
            all_coefficients.append(int(usr_entry))

        # If the input is invalid, warn the user and restart.
        else:
            print(f'Invalid input. Please enter a valid integer or "x" to exit.')
            continue

    return all_coefficients

def main():
    """Main function with user input"""
    
    print(f"This program will store a polynomial and calculate it's x-intercept using newtons method.")
    #some_polynomial = Polynomial(usr_input())
    some_polynomial = Polynomial([6 , 12 , 8 , 1 , 0] , 's(theta)' , 's' , chr(952))

    print(some_polynomial)
    print(some_polynomial.compute(2)[0])
    
    tmp = some_polynomial.derive()

    for i in range(2):
        print(tmp)
        print(tmp.compute(2)[0])
        
        tmp = tmp.derive()

    print(some_polynomial.solve(-0.2 , 6)[0])

main()

This program will store a polynomial and calculate it's x-intercept using newtons method.
s(theta) =  6θ^4 + 12θ^3 + 8θ^2 + 1θ^1 + 0
s(theta) = 226
s'(theta) =  24θ^3 + 36θ^2 + 16θ^1 + 1
s'(theta) = 369
s''(theta) =  72θ^2 + 72θ^1 + 16
s''(theta) = 448
The root found of s(theta) is : -0.160569
